In [28]:
import numpy as np
import pandas as pd
from datetime import datetime
from data import get_indicator_data
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [29]:
df = get_indicator_data("AAPL")

[*********************100%***********************]  1 of 1 completed


Selecting our Predictor Variables from the df

In [30]:
X = df.iloc[:,:9]

Making a df where 1 means tomorrows closing price is higher than todays,<br>
-1 means tomorrows closing price is lower than todays

In [31]:
y = np.where(df['Close'].shift(-1) > df['Close'],1,-1)

splitting dataset into training and test data

In [32]:
split = int(0.7*len(df))
training_X, testing_X, training_Y, testing_Y = X[:split], X[split:], y[:split], y[split:]

fitting LR model

In [33]:
model = LogisticRegression()
model = model.fit (training_X, training_Y)
model_summary = pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))
print(model_summary)

           0                          1
0       Open    [-4.15515809570037e-19]
1       High   [-4.041668141241512e-19]
2        Low   [-3.005205094500823e-19]
3      Close  [-3.9151028081398143e-19]
4  Adj Close   [-4.432370407341504e-19]
5     Volume   [6.8417599039879166e-12]
6        RSI    [2.010106681887889e-17]
7      EMA18  [-1.1746146887481385e-18]
8        OBV   [1.3808796488848106e-11]


In [34]:
start_date = '2024-10-01'
end_date = datetime.today().strftime('%Y-%m-%d') # sets end date to todays date

Create a list containing the start and end dates

In [35]:
prediction_date = [start_date, end_date]

Use the list of dates for prediction

In [36]:
probability = model.predict_proba(testing_X)
print(probability)

[[0.41481514 0.58518486]
 [0.41519587 0.58480413]
 [0.4148243  0.5851757 ]
 [0.41436192 0.58563808]
 [0.41472346 0.58527654]
 [0.41449229 0.58550771]
 [0.41405997 0.58594003]
 [0.413695   0.586305  ]
 [0.41340472 0.58659528]
 [0.41292738 0.58707262]
 [0.4126655  0.5873345 ]
 [0.41235592 0.58764408]
 [0.41199225 0.58800775]
 [0.4116797  0.5883203 ]
 [0.4113735  0.5886265 ]
 [0.4111102  0.5888898 ]
 [0.41142637 0.58857363]
 [0.4119886  0.5880114 ]
 [0.41261612 0.58738388]
 [0.41304725 0.58695275]
 [0.41261826 0.58738174]
 [0.41245636 0.58754364]
 [0.4126765  0.5873235 ]
 [0.41235542 0.58764458]
 [0.41215505 0.58784495]
 [0.41191523 0.58808477]
 [0.4117496  0.5882504 ]
 [0.41139744 0.58860256]
 [0.41046492 0.58953508]
 [0.41106437 0.58893563]
 [0.41074942 0.58925058]
 [0.41050185 0.58949815]
 [0.41082014 0.58917986]
 [0.41051457 0.58948543]
 [0.41003781 0.58996219]
 [0.40914122 0.59085878]
 [0.40907045 0.59092955]
 [0.4086967  0.5913033 ]
 [0.40830457 0.59169543]
 [0.40805555 0.59194445]


Testing model with our testing data

In [37]:
predicted = model.predict(testing_X)

Finding which dates that it predicted to buy<br>
Find the dates with predicted buy signals (Signal = 1)

In [38]:
predicted_labels = 1
buy_signal_dates = prediction_date[predicted_labels == 1]

Print the date(s) with buy signal(s)

In [39]:
print("\nDate(s) with Buy Signal(s):")
print(buy_signal_dates)


Date(s) with Buy Signal(s):
2024-10-30


Reporting model accuracy

In [40]:
print(metrics.classification_report(testing_Y, predicted))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       110
           1       0.63      1.00      0.77       185

    accuracy                           0.63       295
   macro avg       0.31      0.50      0.39       295
weighted avg       0.39      0.63      0.48       295



/opt/homebrew/Caskroom/miniconda/base/envs/quantico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/quantico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/quantico/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.